In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
from os import DirEntry
import sys
import re
from itertools import chain
from numba import jit, njit
from HelperFuncs import *
from convection_param.ConvParamFuncs import *
from convection_param.GetTrainingData import *
import pickle
from tqdm.notebook import tqdm
import datetime
from pprint import pprint
import time

xr.set_options(display_style="html");
%load_ext autoreload
%autoreload 2

This notebooks is for training of different non-deep learning models

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import linear_model
from sklearn import ensemble
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [3]:
# data = np.load('../Processed/TrainData/R2B5_vcg_20221118-153949.npz')
# data = np.load('../local_data/TrainData/20230111-165428-R2B5_y13y16_vcg-fluxes_rho_fluct.npz')
data = np.load('../local_data/TrainData/20230131-171851-R2B5_y13y16_vcg-fluxes_rho_fluct.npz')

print(data.files)

['X_train', 'X_val', 'X_test', 'Y_train', 'Y_val', 'Y_test', 'X_expl', 'Y_expl', 'train_coords', 'val_coords', 'test_coords']


In [4]:
X_train, X_val, X_test, Y_train, Y_val, Y_test, X_expl, Y_expl = \
data['X_train'], data['X_val'], data['X_test'], data['Y_train'], data['Y_val'], data['Y_test'], data['X_expl'], data['Y_expl']

from convection_param.HelperFuncs import unique_unsorted

# vars_to_neglect = ['qr','qi','qs']
# vars_to_neglect = ['qr','qs']
vars_to_neglect = []
vars_to_neglect_mask = ~np.isin(unique_unsorted([e[0] for e in X_expl]), vars_to_neglect)
print(vars_to_neglect_mask)
X_train = X_train[:,:,vars_to_neglect_mask]
X_val = X_val[:,:,vars_to_neglect_mask]
X_test = X_test[:,:,vars_to_neglect_mask]
X_expl = np.array([e for e in X_expl if e[0] not in vars_to_neglect])

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('X_test shape: ', X_test.shape)
print('Y_train shape: ', Y_train.shape)
print('Y_val shape: ', Y_val.shape)
print('Y_test shape: ', Y_test.shape)
print('len X_expl', len(X_expl))
print('len Y_expl', len(Y_expl))

[ True  True  True  True  True  True  True  True  True]
X_train shape:  (1613616, 23, 9)
X_val shape:  (201702, 23, 9)
X_test shape:  (201702, 23, 9)
Y_train shape:  (1613616, 189)
Y_val shape:  (201702, 189)
Y_test shape:  (201702, 189)
len X_expl 207
len Y_expl 189


In [5]:
# If channeled data is input
X_train = X_train.reshape(X_train.shape[0], -1)
# X_train = X_train[~np.isclose(X_train.std(axis=0), 0)]
X_val = X_val.reshape(X_val.shape[0], -1)
# X_val = X_val[~np.isclose(X_val.std(axis=0), 0)]
X_test = X_test.reshape(X_test.shape[0], -1)
# X_test = X_test[~np.isclose(X_test.std(axis=0), 0)]
print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('X_test shape: ', X_test.shape)

X_train shape:  (1613616, 207)
X_val shape:  (201702, 207)
X_test shape:  (201702, 207)


#### Use multiple files

In [6]:
from joblib import dump, load
def save_models(model_list, model_path):
    if os.path.exists(model_path):
        print(f'Path {model_path} already exists')
        return
    os.mkdir(model_path)
    for model in model_list:
        pkl_filename = os.path.join(model_path, get_sk_model_name(model) + '.joblib')
        print(f'Saving to {pkl_filename}')
        dump(model, pkl_filename)

            
def load_models(path, only_paths=False):
    if isinstance(path, list):
        models = []
        for p in path:
            m = load_models(p, only_paths)
            models.extend(m)
    else:
        files = sorted([f for f in os.scandir(path)], key=lambda f: f.name)
        models = []
        files = [file for file in files if file.is_file()]
        for file in files:
            if 'joblib' in file.path:
                if only_paths:
                    print(f'Adding path: {file.path}')
                    models.append(file)
                else:
                    print(f'Loading from file {file.path}')
                    models.append(load(file.path))

    return models

### Regression models

In [7]:
class Baseline:
    def __init__(self):
        self.Y_median = None
    
    def fit(self, X, Y):
        self.Y_median = np.median(Y, axis=0)
    
    def predict(self, X):
        return np.tile(self.Y_median[None,:], (X.shape[0],1))
    
    def __repr__(self):
        return 'Baseline()'

In [8]:
models = [
    # ensemble.RandomForestRegressor(max_features=0.3, min_samples_leaf=0.01, n_jobs=-4),
    linear_model.LinearRegression(),
    ensemble.RandomForestRegressor(n_jobs=-2),
    # linear_model.Ridge(alpha=1),
    linear_model.Ridge(),
    # linear_model.MultiTaskLasso(alpha=1.),
    linear_model.MultiTaskLasso(),
    # linear_model.MultiTaskElasticNet(alpha=1, l1_ratio=0.5),
    linear_model.MultiTaskElasticNet(),
    # KernelRidge(kernel='rbf', alpha=1),
    KernelRidge(),
    # ensemble.ExtraTreesRegressor(max_features=0.3, n_jobs=-4),
    ensemble.ExtraTreesRegressor(n_jobs=-2),
    MultiOutputRegressor(ensemble.GradientBoostingRegressor(), n_jobs=-2),
    MultiOutputRegressor(SVR(), n_jobs=-2),
    # GaussianProcessRegressor(),#ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(50.0, length_scale_bounds="fixed")), # Found by grid search CV on 20% of training data
    # Baseline(),
]

In [9]:
def get_rf_tree_depths(rf):
    return [estimator.get_depth() for estimator in rf]

def get_max_gbr_tree_depth(gbr):
    return max([pred.get_max_depth() for estimator in gbr.estimators_ for predictor_list in estimator._predictors for pred in predictor_list])

In [10]:
from copy import deepcopy
gcv_models = load_models('Models/ClassicHPO/20230419-144242-nsamples143401-5days/')
# gcv_models = load_models('Models/ClassicHPO/20230504-171743-nsamples143401-5days_woqrqs/')
gcv_models = gcv_models[1:]

# gcv_models = [load('Models/ClassicHPO/20230419-144242-nsamples143401-5days/3.joblib')]
models = [deepcopy(m.best_estimator_) for m in gcv_models]

Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/0.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/1.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/2.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/3.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/5.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/6.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/8.joblib
Loading from file Models/ClassicHPO/20230419-144242-nsamples143401-5days/9.joblib


In [11]:
m = models[2]
print(m)
print(m.coef_.size)
print(m.intercept_.size)
print(m.coef_.size + m.intercept_.size)

MultiTaskElasticNet(alpha=0.1, l1_ratio=0.1)
39123
189
39312


In [12]:
max_depth_rf = np.max(get_rf_tree_depths(models[3]))
max_depth_et = np.max(get_rf_tree_depths(models[4]))
max_depth_gbr = get_max_gbr_tree_depth(models[5])

models[3].set_params(max_depth=max_depth_rf)
models[4].set_params(max_depth=max_depth_et)
models[5].estimator.set_params(max_depth=max_depth_gbr)

print(max_depth_rf)
print(max_depth_et)
print(max_depth_gbr)

47
54
27


In [13]:
def get_sk_model_name(model):
    if isinstance(model, DirEntry):
        model = model.path
    model = str(model).split('/')[-1]
    name = model.split('(')[0]
    if name=='MultiOutputRegressor' or name=='GridSearchCV' or name=='RandomizedSearchCV':
        name = re.search('estimator=(.*)\(', str(model)).group(1)
    name = name.replace('.joblib', '')
    return name

print(get_sk_model_name(models[0]))

Ridge


In [14]:
npr.seed(1245845)
sample_frac = 1
sample_idx = npr.choice(X_train.shape[0], size=int(sample_frac*X_train.shape[0]), replace=False)#np.s_[:]#
print(f'Using {len(sample_idx)} samples')

X_train_sample = X_train[sample_idx]
Y_train_sample = Y_train[sample_idx]

Using 1613616 samples


In [ ]:
model_name_list = [get_sk_model_name(model) for model in models]
print(model_name_list)
Y_expl_names = ['_'.join(map(str, expl)) for expl in Y_expl]

path = 'Models/Optimized/'
model_path = os.path.join(path, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")) + f'_{sample_frac}samples'
perform_fit = True
os.makedirs(model_path, exist_ok=True)
fit_times = []
sample_sizes = {}

def comp_metrics(Y_test, Y_test_hat):
    RMSE = np.sqrt(mean_squared_error(Y_test, Y_test_hat))
    R2 = r2_score(Y_test, Y_test_hat, multioutput='raw_values')
    model_var = np.var(Y_test_hat, axis=0)
    return RMSE, R2, model_var

for model, name in tqdm(list(zip(models, model_name_list))[0:3:2]):
    df_RMSE = pd.DataFrame(columns = [name], index = Y_expl_names)
    df_R2 = pd.DataFrame(columns = [name], index = Y_expl_names)
    df_var = pd.DataFrame(columns = [name], index = Y_expl_names)
    if isinstance(model, DirEntry):
        model = load(model)
    print(f'Using Model: {name}')
    
    if perform_fit:
        t0 = time.time()
        print(X_train_sample.shape)
        print(Y_train_sample.shape)
        model.fit(X_train_sample, Y_train_sample)
        fit_time = time.time() - t0
        print(f'fit_time = {fit_time}s')
        fit_times.append(fit_time)
        sample_sizes[name] = len(X_train_sample)
        pkl_filename = os.path.join(model_path, get_sk_model_name(model) + '.joblib')
        print(f'Saving to {pkl_filename}')
        dump(model, pkl_filename)
    
    Y_test_hat = model.predict(X_test)
    
    RMSE, R2, var = comp_metrics(Y_test, Y_test_hat)
    print(f'R2_test({name}): {np.mean(R2)}')
    df_RMSE[name] = RMSE
    df_R2[name] = R2
    df_var[name] = var
    df_RMSE.to_csv(os.path.join(model_path, f'df_RMSE_test_{name}.csv'))
    df_R2.to_csv(os.path.join(model_path, f'df_R2_test_{name}.csv'))
    df_var.to_csv(os.path.join(model_path, f'df_var_test_{name}.csv'))

    Y_val_hat = model.predict(X_val)

    RMSE, R2, var = comp_metrics(Y_val, Y_val_hat)
    print(f'R2_val({name}): {np.mean(R2)}')
    df_RMSE[name] = RMSE
    df_R2[name] = R2
    df_var[name] = var
    df_RMSE.to_csv(os.path.join(model_path, f'df_RMSE_val_{name}.csv'))
    df_R2.to_csv(os.path.join(model_path, f'df_R2_val_{name}.csv'))
    df_var.to_csv(os.path.join(model_path, f'df_var_val_{name}.csv'))

['Ridge', 'MultiTaskLasso', 'MultiTaskElasticNet', 'RandomForestRegressor', 'ExtraTreesRegressor', 'HistGradientBoostingRegressor', 'LassoLars']


  0%|          | 0/2 [00:00<?, ?it/s]

Using Model: Ridge
(1613616, 207)
(1613616, 189)
fit_time = 7.986676931381226s
Saving to Models/Optimized/20230829-100912_1samples/Ridge.joblib
R2_test(Ridge): -2.0760515544344837
R2_val(Ridge): -2.631908964269372
Using Model: MultiTaskElasticNet
(1613616, 207)
(1613616, 189)


In [ ]:
if perform_fit:
    with open(os.path.join(model_path, 'fit_times.csv'), 'w') as f:
        f.write('#model,fit_time\n')
        f.write('#,seconds\n')
        fitted_models_size = len(fit_times)
        for name, fit_time in zip(model_name_list[:fitted_models_size], fit_times):
            f.write(f'{name},{fit_time}\n')
    
    pd.DataFrame(sample_sizes, index=['#samples']).to_csv(os.path.join(model_path, 'sample_sizes.csv'))

In [ ]:
4